# 🚨 Fraud Pattern Mining and JS Rule Generation (Databricks Ready)

In [ ]:

import json
import pandas as pd

def flatten(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, str(v)))
    return dict(items)

# Load fraud JSON file
with open("/dbfs/FileStore/tables/confirmed_fraud_payloads.json", "r") as f:
    data = json.load(f)

# Flatten payloads
flat_data = [flatten(txn['data']['attributes']['activity']['payload']) for txn in data]
df = pd.DataFrame(flat_data).fillna("null")
df.shape


## 🧠 Frequent Pattern Mining (FP-Growth)

In [ ]:

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth

# Convert each row to a transaction of feature=value
transactions = []
for _, row in df.iterrows():
    txn = [f"{col}={val}" for col, val in row.items()]
    transactions.append(txn)

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_tf = pd.DataFrame(te_ary, columns=te.columns_)

# Run FP-Growth
frequent_itemsets = fpgrowth(df_tf, min_support=0.3, use_colnames=True)
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
frequent_itemsets.head(10)


## 🛠️ Generate JavaScript Rules

In [ ]:

# Convert frequent patterns into JS rules
js_rules = []
for _, row in frequent_itemsets.iterrows():
    conditions = []
    for item in row['itemsets']:
        key, val = item.split('=')
        conditions.append(f"transaction['{key}'] === '{val}'")
    rule = "if (" + " && ".join(conditions) + ") { flagAsFraud(); }"
    js_rules.append(rule)

# Output first few rules
for rule in js_rules[:10]:
    print(rule)
